In [67]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import json


def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "C:\Program Files\Common Files\chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

browser = init_browser()
# ------------------------------------------------------------------------------

In [68]:
# Visit website
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

# Get desired fields
news_title = soup.find('div', {"class":"content_title"}).text
news_p = soup.find('div', {"class":"rollover_description_inner"}).text

In [69]:
# ------------------------------------------------------------------------------
# Visit website
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

featured_image_url = soup.find('a', {"id":"full_image"})['data-fancybox-href']
featured_image_url = "https://www.jpl.nasa.gov" + featured_image_url

In [70]:
# ------------------------------------------------------------------------------
# Visit website
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

mars_weather = soup.find('p',\
                         {"class":"TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"})\
                        .getText()
mars_weather = mars_weather.split('hPa',1)[0] + 'hPa'
mars_weather = 'sol' + mars_weather.split('sol', 1)[1]
# ------------------------------------------------------------------------------

In [77]:
# Visit website
url = "https://space-facts.com/mars/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")


table = soup.find("table",{"id":"tablepress-mars"})
ptable = pd.read_html(str(table))[0]

ptable = ptable.set_index(0)
ptable.index.names = [""]
ptable.columns = ["Value"]

html_table = ptable.to_html()

,Value
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [ ]:
# ------------------------------------------------------------------------------
# Visit website
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

sources = soup.find_all("a",{"class":"itemLink product-item"})
links = set(["https://astrogeology.usgs.gov" + link["href"] for link in sources])

hemisphere_image_urls = []
for link in links:
    browser.visit(link)
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    link_dict = {}
    img_link = soup.find('img',{"class":"wide-image"})["src"]
    title = soup.find('h2', {"class":"title"}).text
    link_dict["title"] = title.split(" Enhanced",1)[0]
    link_dict["img_link"] = "https://astrogeology.usgs.gov" + img_link
    
    hemisphere_image_urls.append(link_dict)
# ------------------------------------------------------------------------------

In [65]:
# Store data in a dictionary
mars_data = {
    "news_title": news_title,
    "news_p": news_p,
    "featured_image_url": featured_image_url,
    "mars_weather": mars_weather,
    "html_table": html_table,
    "hemisphere_image_urls": hemisphere_image_urls
}

# Close the browser after scraping
browser.quit()
print(json.dumps(mars_data, indent =4))

{
    "news_title": "NASA Garners 7 Webby Award Nominations",
    "news_p": "Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.",
    "featured_image_url": "https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA10181_ip.jpg",
    "mars_weather": "sol 127 (2019-04-05) low -96.6\u00baC (-141.9\u00baF) high -16.8\u00baC (1.8\u00baF)\nwinds from the SW at 4.2 m/s (9.3 mph) gusting to 11.2 m/s (25.0 mph)\npressure at 7.30 hPa",
    "html_table": "<table border=\"1\" class=\"dataframe\">\n  <thead>\n    <tr style=\"text-align: right;\">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6